# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import torch
import transformers
from tqdm import notebook
import re
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from pymystem3 import Mystem as m
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import numpy
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
import nltk
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


## Подготовка

Откроем датасет и сохраним его в переменную 'data'

In [2]:
try:
    data = pd.read_csv('/datasets/toxic_comments.csv')
except:
    data = pd.read_csv('C:/Users/FunnyFunFruit/Desktop/ЯндексПрактикум/Проекты/Машинное обучение для текстов/toxic_comments.csv',
                  index_col=[0])

Посмотрим строки и размер датасета

In [3]:
data.sample(5)

,text,toxic
83924,""":Thanks for the second award. Now where is Ba...",0
75667,====\n\nVendetta87the name says it allis obvio...,0
123048,Richard is a gay dancing HOBO>HOBO>HOBO!,1
132500,I came over here to suggest the same thing.,0
55223,Warning \n\nWikipedia is not a gameguide. Too ...,0


In [4]:
data.shape

(159292, 2)

Создадим функции для леммтаизации и очистки текста

In [5]:
def get_wordnet_pos(word):
    import nltk
    from nltk import pos_tag
    from nltk.stem import WordNetLemmatizer
    from nltk.corpus import wordnet
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize_text(text):
    def get_wordnet_pos(word):
        import nltk
        from nltk import pos_tag
        from nltk.stem import WordNetLemmatizer
        from nltk.corpus import wordnet
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}
        return tag_dict.get(tag, wordnet.NOUN)
    
    import nltk
    from nltk import pos_tag
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    words = nltk.word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in words]
    lemmatized_text = ' '.join(lemmatized_words)
    return lemmatized_text

def clear_text(text):
    import re
    return ' '.join(re.sub(r"[^a-zA-Z]", ' ', text).lower().split())

Применим функции к датасету

In [6]:
%%time
data['text'] = data['text'].parallel_apply(clear_text)

CPU times: total: 484 ms
Wall time: 1.78 s


In [7]:
%%time
data['text'] = data['text'].parallel_apply(lemmatize_text)

CPU times: total: 10.9 s
Wall time: 9min 37s


In [8]:
data['text'].sample(20)

101174    thanks arabic be a language option but not a c...
4394      something anonymous edit have give rise to van...
146709    do you have a reliable source for these detail...
118029    support all the source call it an islamic cent...
41484                    great article wikipedia jimbo wale
29186     i also ask whether you be give an assessment o...
151483    the only other option to evisceration might be...
80775     no it isn t pathetic i have well thing to do t...
55165     cfd thanks i thought the tool would do this au...
2417      thanks alan thanks for the intro to wiki you d...
73648     for what i never personally attack anyone he b...
113281    note one buzzfeed article have suggest that th...
85556     barrett on corporal clegg some source i ve rea...
115435    why their not heavy metal can t you just accep...
81398                need reference no source unref tag add
79021     i think you have not see that i do reply to yo...
148979    theresa i would urge caution t

Обработаем дисбаланс классов

In [11]:
data_train, data_test = train_test_split(data, test_size=0.4)

print(data_train['toxic'].value_counts())

r = len(data_train.loc[data_train['toxic']==0])//len(data_train.loc[data['toxic']==1])


df_1 = data_train.loc[data_train['toxic']==1]
df_1 = data_train.loc[df_1.index.repeat(r)]
data_train = pd.concat([data_train.loc[data_train['toxic']==0], df_1]).sample(frac=1)

0    85852
1     9723
Name: toxic, dtype: int64


In [12]:
data_train['toxic'].value_counts()

0    85852
1    77784
Name: toxic, dtype: int64

Разделим выборки на обучающую, валидационную и тестовую

In [13]:
features_train = data_train['text']
target_train = data_train['toxic']

features_valid, features_test, target_valid, target_test = train_test_split(data_test['text'], data_test['toxic'], test_size=0.5)

## Обучение

Векторизуем текст через TF-IDF Vectorizer

In [14]:
count_tf_idf = TfidfVectorizer()

In [15]:
tf_idf = count_tf_idf.fit_transform(features_train)

In [16]:
tf_idf_valid = count_tf_idf.transform(features_valid)

In [17]:
tf_idf_test = count_tf_idf.transform(features_test)

Инициализируем модель логистической регрессии

In [18]:
model_1 = LogisticRegression(C=5, max_iter=100000000)

Обучим модель и предскажем токсичность комментариев

In [19]:
%%time

model_1.fit(tf_idf, target_train)

pred_1 = model_1.predict(tf_idf_valid)

print('F1 метрика модели линейной регрессии: {:.3f}'.format(f1_score(target_valid, pred_1)))

F1 метрика модели линейной регрессии: 0.780
CPU times: total: 52.3 s
Wall time: 6.85 s


F1 метрика показала 0.780, что является приемлимым результатом для данной работы по ТЗ.

Попробуем также предстказать токсичность комментариев через классификатор случайного леса

In [20]:
%%time

model_2 = RandomForestClassifier()

params = {'n_estimators' : range(1, 500, 100),
          'max_depth' : range(1, 20, 5),
          'min_samples_leaf' : range(2, 20, 4),
          'min_samples_split' : range(2, 20, 4)}

search = RandomizedSearchCV(model_2, param_distributions=params, scoring='f1', n_jobs=-1)
search.fit(tf_idf, target_train)

model_2 = search.best_estimator_

pred_2 = model_2.predict(tf_idf_valid)

f1_2 = f1_score(target_valid, pred_2)

print('F1 метрика классификационной модели случайного леса: {:.3f}'.format(f1_2))

F1 метрика классификационной модели случайного леса: 0.454
CPU times: total: 13.9 s
Wall time: 6min 20s


Модель классификации случайного леса показала F1 метрику 0.454

Попробуем классифицировать текст через предобученную модель-трансофрмер Martin-ha

Инициализируем токенайзер и саму модель

In [21]:
tokenizer = transformers.AutoTokenizer.from_pretrained('martin-ha/toxic-comment-model')
model_3 = transformers.AutoModelForSequenceClassification.from_pretrained('martin-ha/toxic-comment-model')

Создадим пайплайн из набора пайплайнов библиотеки Transformers

In [22]:
pipeline_3 = transformers.TextClassificationPipeline(model=model_3, tokenizer=tokenizer)

Предскажем токсичность комментариев и посмотрим F1 метрику на 10000 семплов

Для экономии времени возьмем выборку из 10000 сэмплов

In [23]:
data_bert = data.sample(10000)

In [24]:
y_pred = []

for text in notebook.tqdm(data_bert['text']):
    pred = pipeline_3(text[:512])
    y_pred.append(1 if pred[0]['label'] == 'toxic' else 0)
    
f1_bert = f1_score(data_bert['toxic'], y_pred)
print('Метрика F1 модели BERT: {:.3f}'.format(f1_bert))

  0%|          | 0/10000 [00:00<?, ?it/s]

Метрика F1 модели BERT: 0.677


Модель BERT показала метрику 0.677 на 10000 семплов

По итогам валидации лучшей моделью оказалась модель логистической регрессии. Поэтому проверим ее на тестовой выборке

In [25]:
pred_test = model_1.predict(tf_idf_test)
f1_score(target_test, pred_test)

0.7678181023174464

## Выводы

Исходя из проделанного выше исследования следует вывод, что хуже всех с заданием справилась модель классификации случайного леса (0.454). Далее идет модель BERT (F1=0.677), наилучший результат показала модель линейной регрессии (0.767). Также стоит отметить скорость линейной регрессии, модель обучается и предсказывает ответы в течение минуты. В противовес этому стоит сказать, что модель BERT удобна в написании кода, она вся взята из библиотек, предобработка данных идет через стандартный токенизатор, а дальнейшая работа через стандартный пайплайн.<br>
Таким образом выбирать модель следует из того, что нужно конечному потребителю: удобство написания кода или скорость обработки и точность модели. В любом случае BERT, случайный лес и линейная регрессия в данном контексте являются приемлимыми <br>
Стоит также отметить, что до обработки дисбаланса классов случайный лес и модель BERT менялись местами. После обработки дисбаланса у всех моделей поднялась метрика F1